In [13]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "userpass"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.findDocuments({'animal_type':'Dog'}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Center(
        
        html.B(html.H1('CS-340 Dashboard, by Joshua Morris'))),
        html.Div(id='output_div'),
    
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(
        html.A(
            href="https://www.snhu.edu",
            children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
            ]
        )
    ),
    html.Hr(),
        
    html.Div(id='hidden-div', style={'display':'none'}),
    
    dcc.Checklist(
        id="filter_list",
        options=[
            {'label':'Water', 'value':'box_a'},
            {'label':'Mountain/Wilderness', 'value':'box_b'},
            {'label':'Disaster/Tracking', 'value':'box_c'}
            
        ],
        value = []
    ),
    
    
    html.Div(),
    html.Hr(),
    
    #Start datatable div
    html.Div(
        id='data_table_div',
        children=[
        dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),

        row_selectable='single',
        sort_action='native',

        
        page_size=10,
        
        style_data={
            'color':'black',
            'backgroundColor':'white'
        },
        style_data_conditional=[
            {
                'if':{'row_index':'odd'},
                'backgroundColor':'rgb(220, 220, 220)'
            }
        ]
 

                        ),
        ]),
            #End Table Div
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
        id='map-id'
        )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('data_table_div', 'children'),
             [Input('filter_list', 'value')])
def update_dashboard(filters):
    
    
    
    myList = pd.DataFrame.from_records(db.findDocuments({'animal_type':'Dog'}))


    myList.drop(columns=['_id'],inplace=True, errors='ignore')

    
    #Water Rescue
    if ('box_a' in filters):
       myList.query("breed in ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']", inplace=True)
        
        
    #Mountain/Wilderness Rescue    
    if ('box_b' in filters):
       myList.query("breed in ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']", inplace=True)
        
    
    #Disaster/Tracking
    if ('box_c' in filters):
       myList.query("breed in ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']", inplace=True)
    
       
        
       
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in myList.columns]
    data=myList.to_dict('records') 
    
    
    update_graphs(myList)
    
    return (dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in myList.columns],
                         data=myList.to_dict('records'),

        row_selectable='single',
        sort_action='native',

        
        page_size=10,
        
        style_data={
            'color':'black',
            'backgroundColor':'white'
        },
        style_data_conditional=[
            {
                'if':{'row_index':'odd'},
                'backgroundColor':'rgb(220, 220, 220)'
            }
        ]
 

                        ))

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
   Output('graph-id', "children"),
   [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

 
    return [
       dcc.Graph(            
           figure = px.pie(viewData, names='breed', title='Preferred Animals')
       )    
    ]
    

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    # dff.iloc[row,13],dff.iloc[row,14]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row,12],dff.iloc[row,13]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,12],dff.iloc[row,13]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]
#dff.iloc[row,4]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:22084/
